In [ ]:
import pandas as pd
from time import sleep
from unidecode import unidecode
import os
from selenium.webdriver import ActionChains

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from selenium.webdriver.common.alert import Alert


servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [ ]:
link = os.getcwd() + '\index_consulta.html'
navegador.get(link)

In [ ]:
df_processos = pd.read_excel('Processos.xlsx')
df_processos['Status'] = df_processos['Status'].astype(object)

df_processos.dtypes

In [ ]:
for index in df_processos.index:
    aba_inicial = navegador.window_handles[0]

    nome = df_processos['Nome'][index]
    advogado = df_processos['Advogado'][index]
    processo = df_processos['Processo'][index]
    cidade = df_processos['Cidade'][index]


    # escolher o estado e abrir a nova aba
    if cidade.casefold() == 'distrito federal':
        botao = navegador.find_element(By.XPATH, '/html/body/div/div/button')
        ActionChains(navegador).move_to_element(botao).perform()
        sleep(1)
        navegador.find_element(By.XPATH, '/html/body/div/div/div/a[1]').click()
    
    elif cidade.casefold() == 'rio de janeiro':
        botao = navegador.find_element(By.XPATH, '/html/body/div/div/button')
        ActionChains(navegador).move_to_element(botao).perform()
        sleep(1)
        navegador.find_element(By.XPATH, '/html/body/div/div/div/a[2]').click()

    elif unidecode(cidade).casefold() == 'sao paulo':
        botao = navegador.find_element(By.XPATH, '/html/body/div/div/button')
        ActionChains(navegador).move_to_element(botao).perform()
        sleep(1)
        navegador.find_element(By.XPATH, '/html/body/div/div/div/a[3]').click()

    # ir para a nova aba
    nova_aba = navegador.window_handles[1]
    navegador.switch_to.window(nova_aba)

    # preencher as informacoes
    navegador.find_element(By.ID, 'nome').clear()
    navegador.find_element(By.ID, 'nome').send_keys(nome)

    navegador.find_element(By.ID, 'advogado').clear()
    navegador.find_element(By.ID, 'advogado').send_keys(advogado)

    navegador.find_element(By.ID, 'numero').clear()
    navegador.find_element(By.ID, 'numero').send_keys(processo)

    # pesquisar
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()

    # 1° alerta
    alerta = Alert(navegador)
    alerta.accept()

    # 2° alerta
    alerta = None

    while alerta == None:
        try:
            alerta = navegador.switch_to.alert
            status_processo = alerta.text

            if 'sucesso' in alerta.text:
                df_processos.loc[index, 'Status'] = 'True'
            else:
                df_processos.loc[index, 'Status'] = 'False'

            print(f'{nome}: {status_processo}')
            alerta.accept()
        except:
            pass

    # voltar para a aba inicial
    navegador.close()
    navegador.switch_to.window(aba_inicial)

navegador.quit()

df_processos